### RB using Fantasy Pros stats

In [85]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import pandas as pd
pd.options.display.max_columns = None # See all columns
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

The bulk of the preprocessing logic can be found in the source code.

##### To make our dataset, lets pull running backs from the last ~10 years

In [87]:
RB_2021 = football.prepare_RB("2021", True)
RB_2020 = football.prepare_RB("2020", False)
RB_2019 = football.prepare_RB("2019", False)
RB_2018 = football.prepare_RB("2018", False)
RB_2017 = football.prepare_RB("2017", False)
RB_2016 = football.prepare_RB("2016", False)
RB_2015 = football.prepare_RB("2015", False)
RB_2014 = football.prepare_RB("2014", False)
RB_2013 = football.prepare_RB("2013", False)

In [88]:
# RB_2021

In [89]:
all_RB = [RB_2020, RB_2019, RB_2018,
          RB_2017, RB_2016, RB_2015, RB_2014, RB_2013]

RB = pd.concat(all_RB, ignore_index=True)
RB

,ATT,Rushing_Yds,Y/A,LG,20+,Rushing_Td,REC,TGT,Receiving_Yds,Y/R,Receiving_Td,FPTS/G,y,YBCON,YBCON/ATT,YACON,YACON/ATT,BRKTKL,TK LOSS,TK LOSS YDS,LNG TD,10+ YDS,30+ YDS,40+ YDS,50+ YDS,RZ TGT,YACON.1
0,1.000000,1.000000,0.914286,0.959184,1.000000,0.929688,0.214608,0.271612,0.141369,0.419580,0.000000,0.933333,19.6,1.000000,0.657895,1.000000,0.725000,0.901515,0.862500,0.000000,0.959184,0.913043,1.000000,0.875000,0.500000,0.066964,0.228545
1,0.527690,0.490714,0.857143,0.500000,0.462222,0.933333,1.000000,1.000000,1.000000,0.636364,1.000000,0.938095,17.1,0.593721,0.736842,0.400246,0.550000,0.537374,0.560000,0.353005,0.408163,0.547826,0.533333,0.466667,0.000000,1.000000,0.975124
2,0.943311,0.877927,0.857143,0.714286,0.457143,1.000000,0.567986,0.540721,0.511621,0.573427,0.214286,1.000000,15.4,0.967498,0.684211,0.799578,0.600000,1.000000,0.514286,0.606557,0.714286,1.000000,0.571429,0.250000,0.285714,0.153061,0.613006
3,0.654674,0.615364,0.857143,0.632653,0.533333,0.641667,0.433735,0.373832,0.395503,0.580420,0.200000,0.690476,13.3,0.777707,0.789474,0.473198,0.525000,0.424242,0.800000,0.318033,0.632653,0.710145,0.400000,0.700000,0.533333,0.285714,0.398010
4,0.607710,0.622653,0.928571,0.785714,0.647619,0.562500,0.606713,0.630841,0.503118,0.531469,0.428571,0.719048,14.7,0.616780,0.657895,0.627410,0.750000,0.424242,0.600000,0.400468,0.785714,0.565217,0.571429,1.000000,0.857143,0.765306,0.458422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,0.037634,0.098320,0.455172,0.625000,0.300000,0.234375,0.584416,0.694111,0.637899,0.448387,0.535714,0.000000,1.1,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000
748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.180087,0.173077,0.170910,0.393548,0.285714,0.000000,5.0,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000
749,0.018817,0.011201,0.103448,0.112500,0.000000,0.000000,0.103896,0.099159,0.139373,0.503226,0.133929,0.000000,1.4,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000
750,0.025090,0.009245,0.065517,0.050000,0.000000,0.000000,0.371058,0.370879,0.381552,0.425806,0.000000,0.000000,9.3,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000


In [90]:
# Dean's feature selection:
from sklearn.model_selection import train_test_split


y = RB['y']
X = RB.drop(['y', 'FPTS/G'], axis=1)

# Perform feature selection
X_selected, optimal_columns = football.getBestFeatures(X, y)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Get optimal model
model = football.get_model(X_train, y_train)
model

LinearRegression()

In [91]:
y_pred = model.predict(X_test)

In [92]:
from sklearn.metrics import mean_squared_error

# Reset index of y_test
y_test = y_test.reset_index(drop=True)

mse = mean_squared_error(y_test, y_pred)

# Make results df to see visual per player
results = pd.DataFrame()

results['Y_pred'] = y_pred
results['Y_true'] = y_test
results['difference'] = results['Y_true'] - results['Y_pred']

In [93]:
RB_2022 = football.prepare_RB("2022", True)

In [94]:
# Dean's feature selection:
from sklearn.model_selection import train_test_split

X = RB_2022.drop(['y', 'FPTS/G'], axis=1)

X_selected = X[optimal_columns]

y_pred = model.predict(X_selected)

# Make prediction based df for next year

pred_df = pd.DataFrame()

pred_df['Player'] = X['Player']
pred_df['2023_pred'] = y_pred
pred_df['2022_ppg'] = RB_2022['y']

In [95]:
sorted_2023 = pred_df.sort_values(by='2023_pred', ascending=False).reset_index(drop=True)
sorted_2023.head(20)

,Player,2023_pred,2022_ppg
0,Josh Jacobs,18.347052,16.2
1,Derrick Henry,18.326280,16.9
2,Nick Chubb,17.389441,15.0
3,Jamaal Williams,16.086992,12.6
4,Saquon Barkley,15.747816,14.2
5,Miles Sanders,15.591604,11.6
6,Kenneth Walker III,14.342616,11.7
7,Ezekiel Elliott,13.990544,11.3
8,Austin Ekeler,13.793742,15.6
9,Tony Pollard,13.612283,13.1
